Here I have examined a baseline model, that calculates average from the lookback days, and prediction is that calculated average. On the whole training set it is around 0.04, so the goal is to beat it. Model here created manages to get 0.02 val loss. I also used a keras auto tuner for the 1st time.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.2 MB/s 


In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
df.head()

,Opening,High,Low,Closing,Momentum,Range,ohlc
0,2.879833,2.8879,2.8701,2.879821,0.000011,0.0178,2.879414
1,2.884751,2.8941,2.8741,2.884744,0.000007,0.0200,2.884424
2,2.922487,2.9499,2.8787,2.922526,-0.000040,0.0712,2.918403
3,2.924881,2.9462,2.9071,2.924866,0.000015,0.0391,2.925762
4,2.889930,2.9105,2.8762,2.889919,0.000011,0.0343,2.891637


In [ ]:
df_scalled.head(-5)

,Opening,High,Low,Closing,Momentum,Range,ohlc
0,0.142401,0.140186,0.149033,0.142394,0.563639,0.054192,0.142222
1,0.145429,0.143942,0.151533,0.145425,0.554006,0.060890,0.145324
2,0.168665,0.177747,0.154409,0.168688,0.452143,0.216769,0.166357
3,0.170139,0.175506,0.172163,0.170129,0.572368,0.119040,0.170912
4,0.148618,0.153878,0.152846,0.148611,0.563487,0.104427,0.149789
...,...,...,...,...,...,...,...
3532,0.991975,0.974859,1.008314,0.991970,0.486151,0.063965,0.995177
3533,0.990055,0.982037,0.996999,0.990052,0.478911,0.155148,0.993245
3534,0.986508,0.985394,0.998812,0.986473,0.593006,0.163186,0.992761
3535,1.003241,0.993572,1.001106,1.003248,0.439827,0.193113,1.003839


In [ ]:
df_scalled2.head(-5)

,Opening,High,Low,Closing,Momentum,Range,ohlc
0,0.120675,0.117873,0.126956,0.120667,0.690773,0.054192,0.119970
1,0.123241,0.121031,0.129086,0.123235,0.683946,0.060890,0.122587
2,0.142932,0.149455,0.131536,0.142949,0.611761,0.216769,0.140329
3,0.144181,0.147570,0.146659,0.144170,0.696958,0.119040,0.144171
4,0.125944,0.129385,0.130204,0.125935,0.690665,0.104427,0.126353
...,...,...,...,...,...,...,...
3532,0.840629,0.819691,0.858948,0.840612,0.635861,0.063965,0.839473
3533,0.839002,0.825727,0.849309,0.838986,0.630730,0.155148,0.837844
3534,0.835997,0.828549,0.850854,0.835954,0.711583,0.163186,0.837435
3535,0.850176,0.835426,0.852808,0.850169,0.603033,0.193113,0.846780


SCALER1

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
#len(df_numpy[:2500])
scaler1 = scaler.fit(df_numpy[:3000])
df_scalled = scaler1.transform(df_numpy)
#df_scaled_all = scaler.
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
scaler2 = MinMaxScaler()
df_numpy2 = df.to_numpy() 
#len(df_numpy[:2500])
scaler3 = scaler2.fit(df_numpy2)
df_scalled2 = scaler3.fit_transform(df_numpy2)
#df_scaled_all = scaler.
df_scalled2 = pd.DataFrame(df_scalled2, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
lookback = 15
step = 1
delay = 1
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] ##### HIT KURDE 1 to numer kolumny ktora przewiduje wtf
    yield samples, targets

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2501,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2501,
max_index=3001,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3543,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2501 - lookback)
val_steps = (3001 - 2501 - lookback)
test_steps = (3543 - 3001 - lookback)

In [ ]:
print(samples[0][2][3])

0.6897284388542175


Baseline on validation set

In [ ]:
batch_maes = []
for step in range(val_steps):
  samples,targets = next(val_gen)
  for i in range(0,127):
    pred = 0
    for j in range(0,14):
      pred += samples[i][j][3]
    pred /= 15
    mae = np.mean(np.abs(pred - targets[i]))
    batch_maes.append(mae)
print(np.mean(batch_maes))

0.050866267483068675


Baseline on training set

In [ ]:
batch_maes2 = []
for step in range(train_steps):
  samples2,targets2 = next(train_gen)
  for i in range(0,127):
    pred2 = 0
    for j in range(0,14):
      pred2 += samples2[i][j][3]
    pred2 /= 15
    mae2 = np.mean(np.abs(pred2 - targets2[i]))
    batch_maes2.append(mae2)
print(np.mean(batch_maes2))

0.040624508813941675


In [ ]:
print(len(targets))

128


Scaler 1 with cpi and interest

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(128,
  activation='softmax',
  dropout=0.1,
  recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(5,
  activation='softmax',))
model.add(layers.Dense(1))
model.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=400,
  epochs=80,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/80
400/400 [==============================] - 36s 83ms/step - loss: 0.2221 - val_loss: 0.2646
Epoch 2/80
400/400 [==============================] - 33s 81ms/step - loss: 0.1935 - val_loss: 0.2398
Epoch 3/80
400/400 [==============================] - 33s 83ms/step - loss: 0.1307 - val_loss: 0.0906
Epoch 4/80
400/400 [==============================] - 33s 81ms/step - loss: 0.0694 - val_loss: 0.0310
Epoch 5/80
400/400 [==============================] - 33s 83ms/step - loss: 0.0607 - val_loss: 0.0262
Epoch 6/80
400/400 [==============================] - 33s 82ms/step - loss: 0.0553 - val_loss: 0.0249
Epoch 7/80
400/400 [==============================] - 33s 81ms/step - loss: 0.0490 - val_loss: 0.0267
Epoch 8/80
400/400 [==============================] - 32s 81ms/step - loss: 0.0434 - val_loss: 0.0304
Epoch 9/80
400/400 [==============================] - 32s 81ms/step - loss: 0.0398 - val_loss: 0.0259
Epoch 10/80
400/400 [==============================] - 32s 81ms/step - loss: 0.036

KeyboardInterrupt: ignored

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

532/532 [==============================] - 8s 15ms/step - loss: 0.0181


After using fit() and transform() correctly.

In [ ]:
model1 = tf.keras.models.Sequential()
model1.add(layers.LSTM(128,
  activation='softmax',
  dropout=0.1,
  recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model1.add(layers.Dense(5,
  activation='softmax',))
model1.add(layers.Dense(1))
model1.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history1 = model1.fit(train_gen,
  steps_per_epoch=400,
  epochs=80,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/80
400/400 [==============================] - 43s 97ms/step - loss: 0.2141 - val_loss: 0.2807
Epoch 2/80
400/400 [==============================] - 34s 85ms/step - loss: 0.1993 - val_loss: 0.2678
Epoch 3/80
400/400 [==============================] - 34s 86ms/step - loss: 0.1883 - val_loss: 0.2101
Epoch 4/80
400/400 [==============================] - 35s 88ms/step - loss: 0.1340 - val_loss: 0.1315
Epoch 5/80
400/400 [==============================] - 38s 94ms/step - loss: 0.0779 - val_loss: 0.0451
Epoch 6/80
400/400 [==============================] - 36s 90ms/step - loss: 0.0641 - val_loss: 0.0321
Epoch 7/80
400/400 [==============================] - 36s 89ms/step - loss: 0.0582 - val_loss: 0.0270
Epoch 8/80
400/400 [==============================] - 35s 86ms/step - loss: 0.0519 - val_loss: 0.0254
Epoch 9/80
400/400 [==============================] - 35s 87ms/step - loss: 0.0462 - val_loss: 0.0227
Epoch 10/80
400/400 [==============================] - 35s 88ms/step - loss: 0.041

In [ ]:
eval1 = model1.evaluate(test_gen, steps = test_steps)

527/527 [==============================] - 9s 17ms/step - loss: 0.0212


In [ ]:
model2 = tf.keras.models.Sequential()
model2.add(layers.LSTM(units=hp_units,
  activation='softmax',
  dropout=0.1,
  recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model2.add(layers.Dense(5,
  activation='softmax',))
model2.add(layers.Dense(1))
model2.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history2 = model2.fit(train_gen,
  steps_per_epoch=400,
  epochs=80,
  validation_data=val_gen,
  validation_steps=val_steps)

In [ ]:
def model_builder(hp):
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  hp_units = hp.Int('units', min_value=16, max_value=512, step=16)
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(units=hp_units,
    activation=hp.Choice('lstm_activation',values=['relu', 'sigmoid','softmax']),
    recurrent_dropout=0.1,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
  model.add(layers.Dense(5,
    activation=hp.Choice('dense_activation',values=['relu', 'sigmoid','softmax'])))
  model.add(layers.Dense(1))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate), loss='mae')
  return model

In [ ]:
tuner = kt.RandomSearch(model_builder,
                     objective = 'val_loss',
                     #max_epochs=100,
                     #factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt3',
                     max_trials = 30)


INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt3/oracle.json


In [ ]:
#tuner.search(train_gen, val_gen, epochs=100)
tuner.search(train_gen, 
             validation_data = val_gen,
             validation_steps = val_steps, 
             epochs=10,
             steps_per_epoch = 200,
             )


Trial 30 Complete [00h 05m 24s]
val_loss: 0.024721957743167877

Best val_loss So Far: 0.011476642452180386
Total elapsed time: 06h 01m 01s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt3
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.01
units: 352
lstm_activation: sigmoid
Dropout_rate: 0.30000000000000004
dense_activation: sigmoid
Score: 0.011476642452180386
Trial summary
Hyperparameters:
learning_rate: 0.0001
units: 480
lstm_activation: relu
Dropout_rate: 0.1
dense_activation: relu
Score: 0.011646261438727379
Trial summary
Hyperparameters:
learning_rate: 0.001
units: 160
lstm_activation: sigmoid
Dropout_rate: 0.4
dense_activation: relu
Score: 0.012687504291534424
Trial summary
Hyperparameters:
learning_rate: 0.001
units: 144
lstm_activation: relu
Dropout_rate: 0.4
dense_activation: sigmoid
Score: 0.016649186611175537
Trial summary
Hyperparameters:
learning_rate: 0.01
units: 128
lstm_activation: sigmoid
Dropout_rate: 0.0
dense_activation: sigmoid
Score: 0.016744622960686684
Trial summary
Hyperparameters:
learning_rate: 0.01
units: 464
lstm_activation: softmax
Dropout_rate: 0.4
dense_activation: soft

In [ ]:
#best_hps=tuner.get_best_hyperparameters()
best = tuner.get_best_hyperparameters()


In [ ]:
best2 = tuner.get_best_hyperparameters(num_trials =1)

In [ ]:
best[0]

In [ ]:
best2[0]